In [1]:
import argparse as ap

def get_arguments():
    parser = ap.ArgumentParser(prog='./PCR_Deduper.py', formatter_class=ap.RawDescriptionHelpFormatter, description=textwrap.dedent('''\
    PCR Duplicate Remover
    ---------------------
    Removes duplicate reads from given position sorted SAM file 
    that are product of PCR duplication.
    
    Note that the SAM file input is assumed to be pre-sorted by position, and all reads mapped.
    '''))
    parser.add_argument("-f", help="Sorted SAM file to be processed. Must include absolute path the file. <str>", required=True, type=str)
    parser.add_argument("-p", help="If passed as 'True', SAM data is considered paired-ended. <boolean> (def=False)", required=False, type=bool, default=False)
    parser.add_argument("-umi", help="Optional file to define UMI sequences. If not specified, SAM is assumed to use randomers. Must include absolute path to file <str>", required=False, type=str, default='')
    parser.add_argument("-dup_keep", help="Optional argument to designate which read to keep of a duplicate set. 'Q' will keep read with highest quality score, 'F' will keep the first read encountered. No specification will choose a random read. <str>", required=False, type=str, default='')
    #parser.add_argument("-s", help="Set to 'True' for additional summary file output <boolean> (def='False')")
    #parser.add_argument("-h","--help", help="Show this message and exit", required=False, type=str)
    return parser.parse_args()

args = get_arguments()


NameError: name 'textwrap' is not defined

In [ ]:
class sam_info(object):
    """ Filters a sam file line, adjusts alignment start postion based on soft clipping amount, converts quality into phred score, checks for strandeness and unique mapping, gets molecular ID from sam header
    
    Attributes:
        line: single line from a sam file (str)
        start_pos: lines left most mapping postion soft clip adjusted (int)
        qual_score: line quality score (int)
        strand: lines strandeness (str)
        barcode: unique index found in sam header (str)
        
    """
    
    def __init__(self,line):
        """ init
        
        arguments
        line-- a single line from a sorted sam file
        """
        self.line=line
        self.start_pos=self.start_pos() 
        self.qual_score=self.convert_phred()
        self.strand=self.bit_check()
        self.barcode=self.barcode_get()
        
    def soft_clip_check(self,align):
        """Checks for soft clipping and returns soft clip amount cast as an int"""
        soft_amt=0
        if align[1]=="S" or align[1]=="s":
            soft_amt=align[0]
        elif align[2]=="S" or align[2]== "s":
            soft_amt=align[:2]
    
        return(int(soft_amt))

    def convert_phred(self):
        """Converts a quality score into an Phred_33 score and returns the line sum"""
        qual = (self.line.split("\t")[10])
        sum_qual = 0
        for letter in qual:
            n = ord(letter) - 33
            sum_qual+=n
            return sum_qual
        
    def start_pos(self):
        """Returns the left most mapping postion of an alignment adjusted for soft clipping if soft clipping is found"""
        start_pos = int(self.line.split("\t")[3])
        align = self.line.split("\t")[5]
        soft = self.soft_clip_check(align)
        if soft > 0:
            start_pos-=soft
        return start_pos
    
    def barcode_get(self):
        """ looks in the sam header for the (index,barcode,umi, randomer) assumes the barcode is at the end of the header with no spaces 
        and user input barcode length as it is in the header
        ex: NS500451:204:HH7GHBGXY:1:11101:10281:1048:AACCCG -barcode length 6
        NS500451:204:HH7GHBGXY:1:11101:10281:1048-AACCCG^ACCGCN -barcode length 13 (include ^)
        returns the barcode. if no barcode found read will be logged in the log file"""
#         lengthbarcode=6
#         barcode=self.line.split("\t")[0][-6:]
        global COUNT
        barcode=""
        try:
            barcode = subprocess.check_output("cat {sam} |cut -f 1 | grep -v '^@'|sed -n '{NR}'p |grep -o '{seq}'".format(sam=sam,NR=COUNT,seq=seq), shell=True,universal_newlines=True)
            
        except:
            logger.error("Error@ line{co}: Could not find the indetifier in sam header,will not be included in output: {head}\n".format(co=COUNT,head=self.line.split("\t")[0]))
            pass
        COUNT +=1
        
        return barcode.strip("\n")
        
    def bit_check(self):
        """checks that sequence is mapped and and no secondary alignment also gets the strandness (+ or -) *returns strandeness and empty string if uniq mapping*non uniq mapped will be returned but ignored for duplicates (i.e. not output)"""
        flag=int(self.line.split("\t")[1])
        strand="+"
        if ((flag & 4)!=4 and (flag & 256)!=256): #mapped and unique alignment
            umap= ""
        else:
            umap= None
            logger.error("Error@ line{co}: Read is unmapped or non-unique aligned, will not be included in output: {head}".format(co=COUNT,head=self.line.split("\t")[0]))
        if ((flag & 16))==16: #strand is -
            strand="-"
        return (strand,umap)
    
class rmdup(object):
    """ iterates through a sam file, keeping unique non-pcr duplicate reads with best quality"""
    def __init__(self):
        self.sorted_sam=sorted_sam
        """ init
        
        arguments
        sorted_sam: a sam file that has been sorted by alignment postion (i.e. *samtools sort)
        """
        
    
    def iter_sam(sorted_sam):
        
        """ iterates through a sam file and removes pcr duplicates if removal conditions are met"""
         
        global NR
        bestScore=0 # storage for best quality score

        with open(sorted_sam)as sam:


            for line in sam:

                if not line.startswith('@'):
                    line=sam_info(line)
                    NR+=1
                    val=line.barcode,line.start_pos,line.strand[0] #(barcode,start_pos, +or-)

                    if line.strand[1] is None or line.barcode is "": # not mapped or secondary alignment or no index in header *ignore
                        continue
                    elif val in place.keys() and bestScore < line.qual_score : # if val is in dict but current line quality score is highest replace with current line

                        for key in place.keys():
                            if key == val:
                                place[val] = line.line
                            break
                    elif val not in place.keys(): #if values is not in dict put it in with line as value

                        place[val]=line.line
                        bestScore = line.qual_score
                    else: #for cases where duplicates with same quality score 1st read will be retained
                        continue
            return 
        
#class sam_sort(object):
    #""" sorting a sam file"""
    
    #def __init__(self,sam,out):
        
        #""" init
        
        #arguments
        #sam -- a sam file that has not been sorted (str)
        #out -- name of output file (str)
        #"""
        #self.sam=sam
        
        #self.out=out
        
def sort_sam(sam,out):
    """ sorts a sam file by left most mapping postion * defaults to 3M temp memory storage and 28 nodes (see samtools manual -m,-@) may need adjustments based on user system"""
    with tempfile.TemporaryFile() as f: #uses a temp file
        try:
            output=subprocess.check_output("samtools view -bS {sam} |samtools sort -m 3M -@ 28 -o {out} ".format(sam=sam,out=out),shell=True,stderr=subprocess.STDOUT) #runs samtools
        except subprocess.CalledProcessError as er :
            logger.error("error sorting sam file:\n{error}\n".format(error=er.output)) #logs any errors
            exit(1)
                
def file_check(parser, arg):
    """ checks if input files exist"""
    if not os.path.exists(arg):
        parser.error("The file {0} does not exist!".format(arg))
    else:
        return str(arg)